In [ ]:
import pandas as pd
import cv2
import argparse
import numpy as np
import os
from os import listdir
from os.path import isfile, join, isdir

In [ ]:
#mudar o nome da imagem
path = os.getcwd()
os.chdir(path)

local_imagem = (r".\images\Alam. Campinas One_Bed\IMG_8803.jpg")
cfg_teste = ("yolov3.cfg")
weight_teste = ("yolov3.weights")
txt_teste = ("yolov3.txt")

In [ ]:
def get_output_layers(net):
    
    layer_names = net.getLayerNames()
    
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers

def draw_prediction(img, class_id, confidence, x, y, x_plus_w, y_plus_h):

    label = str(classes[class_id])

    color = COLORS[class_id]

    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [ ]:
image = cv2.imread(local_imagem)
Width = image.shape[1]
Height = image.shape[0]
scale = 0.00392
classes = None
with open(txt_teste, 'r') as f:
    classes = [line.strip() for line in f.readlines()]
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))
net = cv2.dnn.readNet(weight_teste, cfg_teste)
blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)
net.setInput(blob)
outs = net.forward(get_output_layers(net))
class_ids = []
confidences = []
boxes = []
conf_threshold = 0.6
nms_threshold = 0.4


In [ ]:
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * Width)
            center_y = int(detection[1] * Height)
            w = int(detection[2] * Width)
            h = int(detection[3] * Height)
            x = center_x - w / 2
            y = center_y - h / 2
            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([x, y, w, h])


indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)


In [ ]:
for i in indices:
    i = i[0]
    box = boxes[i]
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]
    draw_prediction(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))

In [ ]:
for _ in class_ids:
    detected_items = classes[_]
    print(_,detected_items)

In [ ]:
#cv2.imshow("object detection", image)
#cv2.waitKey()
cv2.imwrite("object-detection.jpg", image)
#cv2.destroyAllWindows()